Notebook by Valentin GUILLET

# On the role of neurogenesis in overcoming catastrophic forgetting
*German I. Parisi, Xu Ji, Stefan Wermter, 30 November 2018*

In this notebook, I will present the work of Parisi *et al* and the new network architecture they designed to solve the well-known issue of catastrophic forgetting. I will try to present the results obtained in [this article](https://arxiv.org/abs/1811.02113) presented to NIPS 2018 Workshop on Continual Learning (cf. Bibliography)

## Content

I. [Catastrophic forgetting and Neurogenesis](#part1)<br/>
II. [GWR Networks](#part2)<br/>
1. [The algorithm](#algo)<br/>
2. [A fake dataset](#dataset)<br/>
3. [Implementation](#implem)<br/>
4. [Labeling](#label)<br/>
5. [A real dataset](#iris)<br/>

III. [Overcoming catastrophic forgetting ?](#part3)<br/>
IV. [Deep GWR](#part4)<br/>
[Conclusion](#ccl)<br/>
[Bibliography](#biblio)<br/>

## <a id=part1></a>I. Catastrophic forgetting and Neurogenesis

First of all, we need to understand one of the inherent issue of Neural Networks that the authors try to solve : **catastrophic forgetting**.

For that, let's suppose that we need to solve a classification task with a Deep Neural Network such as recognizing pictures of cats and dogs.
With the current available algorithms, this task is not too hard : we define a NN with a predefined architecture, and then we simply train it on a dataset with pictures of the categories we want to recognize (here, cats and dogs).

This works really well, but imagine now that after that learning process, we want to also recognize pictures of rabbits.
How can we do it ?

- One possible way is to train a new network from scratch with a dataset containing both cats, dogs and rabbits pictures. This works well but we need to relearn from scratch each time we discover a new category and the more categories there are, the longer the training will be.


- Another way is to reuse the same network and train it for a second time on a new dataset containing only rabbit pictures. During this second learning, as we apply gradient descents, the parameters of the network will move towards a configuration that is able to recognize rabbits, but in consequence they will also move away from the previously learned configuration that was able to distinguish between cats and dogs. That means that when we try to learn a new feature, the network forgets what it previously learned : that is what is called **catastrophic forgetting**

To illustrate this second situation, let's imagine that we want to train a robot to move in a house and to interact with its environment. This robot will have to recognize the objects around it in order to take the right actions, but it is impossible to establish in advance a list of every object it will ever see during its life ! As we can't make a dataset with every objects it will see, the only solution we have is to make the robot able to learn continuously while it interacts with the environment : this problematic is called **lifelong learning**.

In summary, the final goal is to make an algorithm able to learn continuously as it gets new experience (**lifelong learning**) while avoiding to overwrite previously learned features (**catastrophic forgetting**).

One interesting approach to solve this difficult task is to get inspired by biology : indeed, the human brain is to a certain extent similar to Neural Networks so it must face the same problematics and, as you didn't forget what you learned during highschool when you arrived in Supaero (well, at least not everything), that means that nature found a way to avoid catastrophic forgetting.

Indeed, based on previous research and many psychological and neurobiological studies, one mechanism that seems important to avoid this is **neurogenesis**, i.e. the generation of new neurons as the brain encounters new situations. The idea is that if no neuron is adapted to the input, then a new neuron needs to be created to fill this gap.
Inspired by this idea, in 2002 Stephen Marsland and his team developped a new network architecture that reproduce this mechanism called **Growing When Required**, or **GWR**.

One main difference between this architecture and classical Neural Networks is that GWR is based on **competitive learning**, a form of unsupervised learning where neurons, called nodes in this setup, compete for the right to respond to a subset of the input data.

## <a id=part2></a>II. GWR Networks

### <a id="algo"></a>1. The algorithm

In this section, I will present the base of Parisi's work : the GWR architecture.

A GWR network is composed of two main elements : nodes and edges.
A **node** is a point in the input space and the objective of the network is to learn a set of nodes that represent well the distribution of the input space.

Nodes that are close together are linked by **edges** to form neighbourhoods of nodes that represent similar perception.
Each edge has an *age* that corresponds to the number of iterations since it hasn't been activated.

During the learning, the nodes are going to move in the input space to best correspond to the input distribution.
When seeing a new sample, we select the closest node to this sample, called the **Best Matching Unit (BMU)** and if the distance between this node and the sample is below a threshold, we move the BMU and its neighbours towards the sample to better represent it.
In the case where no node is close enough to the sample, that means that our set of nodes is still distant from the input distribution and we add a new node to our set (it is the **neurogenesis**).

Each node has a **firing counter**, initialized to 1 and that can only decrease, that indicates if this node fired a lot in its life or not.

When we add a new node to our network, we want to be sure to leave him the necessary time to learn the input distribution before adding another node. One way is to give a **firing counter** to each node, initialized to 1 and that decreases each time the associated node is the BMU of a sample. This value indicates the number of times the node was useful to represent the input distribution, and so we add a new neuron only if the firing counter of the BMU is low enough.

Each time a node is determined as BMU, its firing counter and the firing counter of each of its neighbour is decreased following $$\textrm{firing_counter} \leftarrow \textrm{firing_counter} + \tau_i * \kappa * (1 - \textrm{firing_counter}) - \tau_i$$ with $\kappa$, $\tau_i = \tau_{\mathrm{BMU}}$ for the BMU and $\tau_i = \tau_{\mathrm{neighbour}}$ for the BMU's neighbours three constants.

So the learning goes as follow :
1. Sample a point $x$ from the input dataset


2. For each node $w_i$ in the network, compute the distance between it and the sample $d_i = \lVert w_i - x \rVert$


3. Select the two nodes that have the smallest distance. The closest one is called Best Matching Unit (BMU) : $$\mathrm{BMU} = \underset{i \in \mathrm{Nodes}}{\operatorname{argmin}}{\lVert w_i - x \rVert}$$ $$\mathrm{Second} = \underset{i \in \mathrm{Nodes} / \{\mathrm{BMU}\}}{\operatorname{argmin}}{\lVert w_i - x \rVert}$$


4. Create an edge between these two closest nodes if they were not linked before, and reset the age of this edge to zero $\mathrm{Edge} = \mathrm{Edge} \cup \{(\mathrm{BMU}, \mathrm{Second})\}$


5. If $\exp(-d_{\mathrm{BMU}}) < \alpha_t$ and $\mathrm{BMU}.firing counter < h_t$ (i.e. the BMU is too far from the sample and has been added several time steps before)
        a) Add a new node to the network in the middle of the BMU and the sample
$$w_{new} = \frac{w_{\mathrm{BMU}} + x}{2}$$
        b) Connect the new node to the BMU and the second best unit
$$\mathrm{Edge} = (\mathrm{Edge} / \{(\mathrm{BMU}, \mathrm{Second})\}) \cup \{(\mathrm{BMU}, \mathrm{new}), (\mathrm{Second}, \mathrm{new})\}$$


    Else:
        Move the BMU and its neighbours towards the sample
$$w_{\mathrm{BMU}} += \epsilon_{\mathrm{best}} * \mathrm{BMU}.firing counter * (x - w_{\mathrm{BMU}})$$
$$\forall neigh \in neighbours(\mathrm{BMU}),$$
$$w_{\mathrm{neigh}} += \epsilon_{\mathrm{neigh}} * \mathrm{neigh}.firing counter * (x - w_{\mathrm{neigh}})$$


6. Increase the age of every edge connected to the Best Matching Unit by 1


7. Slightly decrease the firing counter of the BMU and its neighbours


8. Remove every edge that is older than some constant $\mathrm{max\_age}$


9. Remove every node that doesn't have any edge

with :
* $\alpha_t$ : the activity threshold corresponding to the minimal distance between the BMU and the sample that leads to adding a new node
* $h_t$ : the firing threshold corresponding to the maximal firing counter of the BMU that leads to adding a new node
* $\epsilon_{\mathrm{best}}$ : a coefficient controlling the amplitude of the move of the BMU towards the sample
* $\epsilon_{\mathrm{neigh}}$ : a coefficient controlling the amplitude of the move of every neighbour of the BMU towards the sample

To apply this algorithm, we start by initializing two random nodes in the input space and then the network automatically adjusts these nodes towards the input distribution, and when the existing nodes are not sufficient to describe correctly the data, it automatically creates new nodes.

### <a id="dataset"></a>2. A fake dataset : artificial flowers

To be sure to understand, let's code a quick example.
Let's imagine a fake dataset of flowers that live in three dimensions (so each flower is represented by three characteristics). Let's suppose that we have 200 flowers and 4 categories of flowers.

To simulate that, we're gonna generate 200 random 3D-points around 4 random "representative flowers".

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
%matplotlib

from sklearn.metrics import confusion_matrix

Using matplotlib backend: TkAgg


In [2]:

def generate_dataset(nb_points=200, nb_labels=4, var=0.1):

    # First, we select where we want our "representative flowers" to be
    centers = []
    while len(centers) < nb_labels:
        new_center = np.random.rand(3)
        # We make sure that these "representative flowers" are not to similar to be able to distinguish them
        if all([np.linalg.norm(center - new_center) > 0.5 for center in centers]):
            centers.append(new_center)


    # Then, we generate some fake flowers with a normal distribution around these "representative flowers"
    #   and we label each of them with their category
    points = []
    labels = []
    for i in range(nb_points):
        center = centers[i % nb_labels]
        point = np.random.normal(center, var)
        points.append(point)
        labels.append(i % nb_labels)
        
    idx = np.random.permutation(nb_points)

    return np.array(points)[idx], np.array(labels)[idx], np.array(centers)

fake_flowers, categories, representative_flowers = generate_dataset()

Let's visualize this dataset :

In [3]:
%matplotlib

def plot_dataset(points, labels, centers=None):

    fig = plt.figure("Data", figsize=(20, 20))
    ax = fig.add_subplot(111, projection='3d')

    # Plot every fake flower in our dataset
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=labels)
    
    # Plot also the "representative flowers"
    if centers is not None:
        ax.scatter(centers[:, 0], centers[:, 1], centers[:, 2], s=150, c='red')

    plt.show()

plot_dataset(fake_flowers, categories, representative_flowers)

Using matplotlib backend: TkAgg


We can see that we have four "representative flowers" in big red dots, and 200 flowers that each belongs to one of the categories of flower while still being slightly different from each other. Our objective is, given this dataset without the "representative flowers", to find a discrete representation of this space.

**NB :** A good representation of this dataset is obviously the four "representative flowers" used to generate every other fake flower, so in the end we would like to find something similar to these four points

### <a id="implem"></a>3. Implementation of GWR

First of all, let's define the structure of Node.
This class is very basic : it contains a vector called `weights` representing the node in the input space initialized randomly, and a firing counter initialized to 1.

We also implement three methods : one that, given a sample in the input space, returns the distance between this sample and the node, another to update the weights of the node and the last one to update the firing counter.

In [4]:

class Node:

    def __init__(self, input_size, weights=None):
        if weights is None:
            self.weights = np.random.rand(input_size)
        else:
            self.weights = weights

        self.firing_counter = 1

    def compute_distance(self, input_vector):
        """Return the distance between the node and an input vector."""
        return np.linalg.norm(self.weights - input_vector)**2
    
    def update_weights(self, input_vector, bmu=False):
        """Update the weights of the node depending on whether it is the BMU or a neighbour of the BMU."""
        epsilon = EPS_BMU if bmu else EPS_NEIGH
        self.weights += epsilon * self.firing_counter * (input_vector - self.weights)

    def update_firing_counter(self, bmu=False):
        """Update the firing counter of the node depending on whether it is the BMU or a neighbour of the BMU."""
        tau = TAU_BMU if bmu else TAU_NEIGH
        self.firing_counter += tau * KAPPA * (1 - self.firing_counter) - tau

In [5]:
# Let's test this class by plotting nodes with our fake dataset

# Generate 10 random nodes
nodes = np.array([Node(3).weights for i in range(10)])

# Plot these points in our input space
plot_dataset(fake_flowers, categories, nodes)

Then, let's implement the edge class.
This class contains :
* `edges` : the adjacency matrix of the network (i.e. if `edges[node1][node2]` == 1, it means that node1 and node2 are connected)
* `ages` : the age of each of these edges

We also implement many simple and useful methods to add or remove an edge between two nodes, update the age of edges of the BMU, get every neighbour of a given node or remove a node from the network.

**NB :** the adjacency matrix is by definition symetric, we could store it as a triangular matrix for performance reasons but this would complicate the code for just a small gain. Instead, we define two methods `set_edge` and `set_age` to modify the values in the matrix symetrically

In [6]:

class Edges:

    def __init__(self, nb_nodes):
        self.edges = np.zeros((nb_nodes, nb_nodes))
        self.ages = np.zeros((nb_nodes, nb_nodes))

    def set_edge(self, index1, index2, value):
        self.edges[index1, index2] = value
        self.edges[index2, index1] = value

    def set_age(self, index1, index2, value):
        self.ages[index1, index2] = value
        self.ages[index2, index1] = value
        
    def reset_edge(self, best_index, second_index):
        """Sets an edge between the BMU and the Second BMU with an age of 0"""
        self.set_edge(best_index, second_index, 1)
        self.set_age(best_index, second_index, 0)

    def add_edge(self, best_index, second_index):
        """Add a new column to the edge matrix (in case of a new node) and links the new
        node to the BMU and the Second BMU"""
        n = self.edges.shape[0]
        # Add new columns to edges and ages
        tmp = self.edges
        self.edges = np.zeros((n+1, n+1))
        self.edges[:-1, :-1] = tmp
        tmp = self.ages
        self.ages = np.zeros((n+1, n+1))
        self.ages[:-1, :-1] = tmp

        # Remove the link between the BMU and the Second BMU
        self.set_edge(best_index, second_index, 0)
        self.set_age(best_index, second_index, 0)
        
        # Link the BMU and the new node, and the Second BMU to the new node
        self.set_edge(best_index, n, 1)
        self.set_edge(second_index, n, 1)
        
    def get_neighbours(self, node_index):
        """Return all the neighbours of a given node"""
        return np.nonzero(self.edges[node_index])[0]
        
    def update_age(self, best):
        """Update the age of every edge linked to the BMU"""
        neighbours = self.get_neighbours(best)
        for neighbour in neighbours:
            age = self.ages[best, neighbour]
            self.set_age(best, neighbour, age+1)

    def remove_edges(self):
        """Remove every edge that is older than a threshold"""
        for i in range(self.edges.shape[0]):
            for j in range(i+1, self.edges.shape[0]):
                if self.ages[i, j] > MAX_AGE:
                    self.set_edge(i, j, 0)
                    self.set_age(i, j, 0)

    def remove_node(self, node_index):
        """Remove a column and a line from edges and ages corresponding to the node"""
        self.edges = np.delete(self.edges, node_index, axis=0)
        self.edges = np.delete(self.edges, node_index, axis=1)
        self.ages = np.delete(self.ages, node_index, axis=0)
        self.ages = np.delete(self.ages, node_index, axis=1)


We now have the two simple data structures we need for the GWR algorithm.
What we need to do now is simply to define our hyperparameters and to follow the algorithm step by step.

In [7]:
ALPHA_T = 0.70     # Insert threshold
H_T     = 0.10     # Firing threshold

EPS_BMU   = 1      # BMU learning rate
EPS_NEIGH = 0.05   # BMU's neighbours learning rate

KAPPA     = 1.05   # General firing counter update constant
TAU_BMU   = 0.3    # BMU firing counter update constant
TAU_NEIGH = 0.1    # BMU's neighbours firing counter update constant

MAX_AGE = 25       # Maximum age of an edge

In [8]:

class Network:

    def __init__(self, input_size):
        self.input_size = input_size

        # We start with only two nodes in our network
        self.nb_nodes = 2
        self.nodes = []
        for i in range(self.nb_nodes):
            self.nodes.append(Node(input_size))

        self.edges = Edges(self.nb_nodes)

    def find_bmus(self, input_vector):
        """Return the index of the two nodes which are the closest to the input vector (the BMU and the second BMU)
        and the distance BMU-input vector."""
        distance = np.array([node.compute_distance(input_vector)
                             for node in self.nodes])
        indexes = np.argsort(distance)
        return indexes[0], indexes[1], distance[indexes[0]]

    def add_node(self, best_index, second_index, input_vector):
        """Add a new node linked to the BMU and the Second BMU, and located in the middle
        of the BMU and the input vector."""
        new_weights = (input_vector + self.nodes[best_index].weights) / 2
        new_node = Node(self.input_size, new_weights)
        self.nodes.append(new_node)
        self.edges.add_edge(best_index, second_index)
        self.nb_nodes += 1

    def remove_nodes(self):
        """Remove a node and the corresponding edges"""
        for node in range(self.nb_nodes-1, -1, -1):
            if len(self.edges.get_neighbours(node)) == 0:   # if the node doesn't have any neighbour
                self.edges.remove_node(node)
                self.nodes.pop(node)
                self.nb_nodes -= 1

    def fit(self, samples, epochs=10):
        """Apply the GWR algorithm and return a list of the evolution of the nodes in the input space
        and the index of the BMU at each step"""

        weights, indexes = [], []
        for epoch in range(epochs):
            for sample in samples:

                # Step 1 : Find the BMU and the second BMU
                bmu, second, bmu_dist = self.find_bmus(sample)

                # Step 2 : Add an edge between the BMU and the second and set its age to 0
                self.edges.reset_edge(bmu, second)

                # Step 3 : If the bmu is too far from the sample and it has already fired a lot in its life :
                if np.exp(-bmu_dist) < ALPHA_T and self.nodes[bmu].firing_counter < H_T:
                    # Then add a new node in the network
                    self.add_node(bmu, second, sample)

                else:
                    # Else, just move the BMU and its neighbours closer to the sample
                    self.nodes[bmu].update_weights(sample, bmu=True)
                    for neighbour in self.edges.get_neighbours(bmu):
                        self.nodes[neighbour].update_weights(sample)

                # Step 4 : update the firing counter of the BMU and of its neighbours
                self.nodes[bmu].update_firing_counter(bmu=True)
                for neighbour in self.edges.get_neighbours(bmu):
                    self.nodes[neighbour].update_firing_counter()

                # Step 5 : each edge connected to the BMU gets older
                self.edges.update_age(bmu)
                # Step 6 : if an edge is too old, remove it
                self.edges.remove_edges()
                # Step 7 : if a neuron doesn't have any edges, remove it
                self.remove_nodes()

                weights.append([node.weights.copy() for node in self.nodes])
                indexes.append(bmu)

        return weights, indexes


Now we just need to instance this network and make it run :

In [9]:
input_size = fake_flowers.shape[1]
network = Network(input_size)

weights, indexes = network.fit(fake_flowers, 1)
print(f"Final number of nodes : {network.nb_nodes}")

Final number of nodes : 4


Let's define a small function to observe what the algorithm is really doing :

In [10]:

def animate(points, labels, weights, indexes):
    
    centers = weights[:]
    maxi_len = len(max(centers, key=len))
    for i in range(len(centers)):
        while len(centers[i]) < maxi_len:
            centers[i].append(centers[i][-1])
        centers[i] = np.array(centers[i])
    centers = np.array(centers)
    size = centers.shape[1]

    def update(num, nodes):
        nodes._offsets3d = (centers[num][:, 0], centers[num][:, 1], centers[num][:, 2])
        col = ['blue'] * size
        col[indexes[num]] = 'red'
        nodes._facecolor3d = col
        return nodes,

    fig = plt.figure("Animation")
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=labels)
    nodes = ax.scatter(centers[0][:, 0], centers[0][:, 1], centers[0][:, 2], s=200, c='blue')
    points_anim = animation.FuncAnimation(fig, update, len(centers), fargs=(nodes,),
                                          interval=1, repeat=False)
    return fig, points_anim


In [11]:
fig, _ = animate(fake_flowers, categories, weights, indexes)
fig.show()

At each time step, we display the position of the nodes in the input space with a big dot which is blue except for the BMU where it is red.

We can see that in the beginning, there are only two nodes that quickly moves toward a flower category. But as there are not enough nodes for each category, the distance between a node and a random sample quickly becomes big, so the network creates a new node. At the end of the training, the network has 4 nodes which are in the center of each of the fake flower distributions, so it is a good representation of the dataset.

### <a id="label"></a>4. Labeling

This algorithm works well but in practice, we want to be able to categorize the data with it. Fortunately, this requires only a very simple modification of the basic algorithm : the addition of a label matrix.

The idea is simple : each node stores the number of times that a given sample label $l$ has been associated to its neural weight (i.e. it has been designated BMU).
So initially, each node store a frequency of 1 for all the label that exist, and then if a node is designated BMU for a sample with label $l$, it will increase its frequency associated to $l$ and decrease the other frequency.

In the end, each node stores the frequency of each label, and so we simply associate the node with the label with the biggest frequency.
From that, to categorize a new input data point, we simply compute the BMU of this sample and the prediction is the label of this BMU.

Implementation wise, we add an attribute `labels` to the node which is a dictionnary associating each label to its frequency.

In [12]:

class LabelizedNode:

    def __init__(self, input_size, weights=None):
        if weights is None:
            self.weights = np.random.rand(input_size)
        else:
            self.weights = weights

        self.labels = {}
        self.firing_counter = 1

    def compute_distance(self, input_vector):
        """Return the distance between the node and an input vector."""
        return np.linalg.norm(self.weights - input_vector)**2
    
    def update_weights(self, input_vector, bmu=False):
        """Update the weights of the node depending on whether it is the BMU or a neighbour of the BMU."""
        epsilon = EPS_BMU if bmu else EPS_NEIGH
        self.weights += epsilon * self.firing_counter * (input_vector - self.weights)

    def update_firing_counter(self, bmu=False):
        """Update the firing counter of the node depending on whether it is the BMU or a neighbour of the BMU."""
        tau = TAU_BMU if bmu else TAU_NEIGH
        self.firing_counter += tau * KAPPA * (1 - self.firing_counter) - tau

    def add_label(self, label):
        """Add a new label that we never saw before and initialize its frequency to 1."""
        self.labels[label] = 1

    def copy_labels(self, labels):
        """Initialize the label dictionnary with every label that has been encountered before."""
        for key in labels.keys():
            self.labels[key] = 1

    def update_labels(self, label):
        """Update the frequency of the label that this node has been associated with."""
        self.labels[label] += 1
        for l in self.labels.keys():
            if l != label:
                self.labels[l] -= 0.1

    def get_label(self):
        """Get the label with the largest frequency : it is the label associated with this node."""
        return max(self.labels.keys(), key=lambda k:self.labels[k])

In [13]:

class LabelizedNetwork:

    def __init__(self, input_size):
        self.input_size = input_size

        # We start with only two nodes in our network
        self.nb_nodes = 2
        self.nodes = []
        for i in range(self.nb_nodes):
            self.nodes.append(LabelizedNode(input_size))

        self.edges = Edges(self.nb_nodes)

    def find_bmus(self, input_vector):
        """Return the index of the two nodes which are the closest to the input vector (the BMU and the second BMU)
        and the distance BMU-input vector."""
        distance = np.array([node.compute_distance(input_vector)
                             for node in self.nodes])
        indexes = np.argsort(distance)
        return indexes[0], indexes[1], distance[indexes[0]]

    def add_node(self, best_index, second_index, input_vector, label):
        """Add a new node linked to the BMU and the Second BMU, and located in the middle
        of the BMU and the input vector."""
        new_weights = (input_vector + self.nodes[best_index].weights) / 2
        new_node = LabelizedNode(self.input_size, new_weights)
        # We initialize the labels of the new node to 1
        new_node.copy_labels(self.nodes[0].labels)
        # We increase the frequency of the label associated to the new node
        new_node.labels[label] += 1
        self.nodes.append(new_node)
        self.edges.add_edge(best_index, second_index)
        self.nb_nodes += 1

    def remove_nodes(self):
        """Remove a node and the corresponding edges"""
        for node in range(self.nb_nodes-1, -1, -1):
            if len(self.edges.get_neighbours(node)) == 0:   # if the node doesn't have any neighbour
                self.edges.remove_node(node)
                self.nodes.pop(node)
                self.nb_nodes -= 1

    def fit(self, samples, labels, epochs=10):
        """Apply the GWR algorithm and return a list of the evolution of the nodes in the input space
        and the index of the BMU at each step"""

        weights, indexes = [], []
        for epoch in range(epochs):
            for sample, label in zip(samples, labels):
                
                # If the label has never been seen before, we add it to each node with a frequency of 1
                if label not in self.nodes[0].labels:
                    for node in self.nodes:
                        node.add_label(label)

                # Step 1 : Find the BMU and the second BMU
                bmu, second, bmu_dist = self.find_bmus(sample)

                # Step 2 : Add an edge between the BMU and the second and set its age to 0
                self.edges.reset_edge(bmu, second)

                # Step 3 : If the bmu is too far from the sample and it has already fired a lot in its life :
                if np.exp(-bmu_dist) < ALPHA_T and self.nodes[bmu].firing_counter < H_T:
                    # Then add a new node in the network
                    self.add_node(bmu, second, sample, label)

                else:
                    # Else, just move the BMU and its neighbours closer to the sample
                    self.nodes[bmu].update_weights(sample, bmu=True)
                    for neighbour in self.edges.get_neighbours(bmu):
                        self.nodes[neighbour].update_weights(sample)
                    self.nodes[bmu].update_labels(label)

                # Step 4 : update the firing counter of the BMU and of its neighbours
                self.nodes[bmu].update_firing_counter(bmu=True)
                for neighbour in self.edges.get_neighbours(bmu):
                    self.nodes[neighbour].update_firing_counter()

                # Step 5 : each edge connected to the BMU gets older
                self.edges.update_age(bmu)
                # Step 6 : if an edge is too old, remove it
                self.edges.remove_edges()
                # Step 7 : if a neuron doesn't have any edges, remove it
                self.remove_nodes()

                weights.append([node.weights.copy() for node in self.nodes])
                indexes.append(bmu)

        return weights, indexes
    
    def predict(self, samples):
                
        labels = []
        for sample in samples:
            bmu, _, _ = self.find_bmus(sample)
            labels.append(self.nodes[bmu].get_label())
        return np.array(labels)


We can now test if the entire algorithm is working but first learning the input distribution on a training dataset and then make some prediction on a testing dataset and compute the generalization error :

In [14]:
def test_GWR(dataset, labels, disp_anim=True, epochs=1, network=None):
    n, input_size = dataset.shape

    train_data, train_labels = dataset[:2*n//3], labels[:2*n//3]
    test_data, test_labels = dataset[2*n//3:], labels[2*n//3:]

    if network is None:
        network = LabelizedNetwork(input_size)

    weights, indexes = network.fit(train_data, train_labels, epochs)
    print(f"Number of nodes : {network.nb_nodes}")

    train_err = 100 - (network.predict(train_data) == train_labels).sum() / (2 * n // 3) * 100
    test_err  = 100 - (network.predict(test_data ) == test_labels ).sum() / (n // 3) * 100
    print(f"Train error : {train_err:.4}%\n"
          f"Generalization error : {test_err:.4}%")

    print("Confusion matrix :\n", confusion_matrix(test_labels, network.predict(test_data)))

    if disp_anim:
        return animate(train_data, train_labels, weights, indexes)
    
    else:
        return network


In [15]:
fig, _ = test_GWR(fake_flowers, categories)
fig.show()

Number of nodes : 4
Train error : 0.0%
Generalization error : 1.515%
Confusion matrix :
 [[18  0  0  0]
 [ 0 16  0  0]
 [ 2  0 15  0]
 [ 0  0  0 16]]


It is clear that each node represents very well a type of flower and that the labeling is very easy because of the non-overlapping categories.
Of course, these data were simple but we can make the training more difficult :

In [16]:
fake_flowers2, categories2, representative_flowers2 = generate_dataset(nb_points=1000, nb_labels=7, var=0.16)
plot_dataset(fake_flowers2, categories2, representative_flowers2)

In [17]:
fig, _ = test_GWR(fake_flowers2, categories2)
fig.show()

Number of nodes : 10
Train error : 9.459%
Generalization error : 9.009%
Confusion matrix :
 [[45  0  1  1  0  1  0]
 [ 1 46  0  0  0  0  5]
 [ 4  1 44  0  0  0  0]
 [ 1  0  1 41  0  3  0]
 [ 0  0  0  0 38  7  0]
 [ 1  0  0  0  1 45  0]
 [ 0  3  0  0  0  0 44]]


We can see that when the categories start to overlap, the network creates more nodes than the number of categories to try to be more precise, but in spite of that, the performance is still affected. Nevertheless, the resulting nodes are are a good approximation of the input space distribution, so it shows that the algorithm is efficient.

### <a id="iris"></a>5. A real dataset

We can't conclude on the efficiency of the algorithm without first trying it on the Iris dataset, so let's do it :

In [19]:
from dataset.load_iris import get_dataset

flowers, iris_labels = get_dataset()
fig, _ = test_GWR(flowers, iris_labels)
fig.show()

Number of nodes : 3
Train error : 10.0%
Generalization error : 10.0%
Confusion matrix :
 [[17  0  0]
 [ 0 15  1]
 [ 0  4 13]]


As predicted, the GWR creates 3 nodes that each correspond to a category of flowers. We can also see that the algorithm doesn't make any mistake on the prediction of the category 1 (which is quite different from the two other categories), but mix up a few flowers in the two categories that are overlapping.

**NB :** the iris flowers are represented by a 4D vector, the 3D representation is simply a projection of the flowers in the 3D space, so some information is lost in the process

## III. <a id=part3></a>Overcoming catastrophic forgetting ?

One important thing we need to check is the assertion that neurogenesis can overcome catastrophic forgetting. For that, let's first generate 5 categories of flowers and let's train a network on only 4 of these categories :

In [20]:
all_flowers, all_categories, all_centers = generate_dataset(nb_points=750, nb_labels=5, var=0.13)
not_last_category, last_category = (all_categories != 4), (all_categories == 4)
fake_flowers_not5, fake_flowers_5 = all_flowers[not_last_category], all_flowers[last_category]
categories_not5, categories_5 = all_categories[not_last_category], all_categories[last_category]
centers_not5, centers_5 = all_centers[:-1], all_centers[-1]

plot_dataset(fake_flowers_not5, categories_not5, centers_not5)

network = test_GWR(fake_flowers_not5, categories_not5, disp_anim=False, epochs=5)

Number of nodes : 4
Train error : 2.25%
Generalization error : 2.0%
Confusion matrix :
 [[47  0  0  0]
 [ 0 47  1  0]
 [ 0  0 44  0]
 [ 1  0  2 58]]


Now let's train the network just on the last category :

In [21]:
fig, _ = test_GWR(fake_flowers_5, categories_5, network=network)
fig.show()

Number of nodes : 5
Train error : 7.0%
Generalization error : 8.0%
Confusion matrix :
 [[ 0  0  0  0]
 [ 0  0  0  0]
 [ 0  0  0  0]
 [ 1  1  2 46]]


We observe that immediatly at the start of the training, as the data are quite different from the learned nodes of the network, the algorithm creates a new node in the area of the 5th category of flowers. So after that, the other nodes aren't really affected by the learning process and only the new node is learning the distribution of the new data.

In [22]:
test_err  = 100 - (network.predict(all_flowers) == all_categories).sum() / (all_categories.shape[0]) * 100
print(f"Generalization error : {test_err:.4}%")

print("Confusion matrix :\n", confusion_matrix(all_categories, network.predict(all_flowers)))

plot_dataset(all_flowers, all_categories, np.array([node.weights for node in network.nodes]))

Generalization error : 4.8%
Confusion matrix :
 [[149   0   0   1   0]
 [  0 147   3   0   0]
 [  2   7 140   1   0]
 [  8   0   3 139   0]
 [  0   1   2   8 139]]


So we can see that even after retraining the network on completely different data, it didn't forget what it had learn previously ! The catastrophic forgetting is indeed overcome *via* neurogenesis.

## <a id=part4></a>IV. Deep GWR

In their NIPS paper, Parisi *et al.* developped a more complex architecture based on the GWR networks.
Their idea is, as in Deep Neural Networks, to use several layers of GWR to be able to reduce again the dimensionality of the input. The architecture is the following :
* The first layer of the network is a GWR that takes an image as input and that learns to represent the input space with a set of nodes. So these nodes correspond to a discretized version of the input space, simpler and that might be more meaningful.
* Then, a second GWR network receives as input the weight distribution of the Best Matching Unit of the first layer. So this second layer learns an even simpler representation of the input space as it learns a representation of its representation.
* After as many layers as necessary, the last layer of the network is a labelized GWR (called Associative GWR or AGWR) that associates an input to a learned category

Exactly as in the case of Deep Learning, the objective is to get finer and finer descriptions of the input space with the several GWR layers, so that the classification in the last layer is easier.

The authors tested this Deep GWR architecture on three different datasets and they showed that in the three cases, the performances of DGWR were better than those of classic deep Neural Networks.
On a classification task on ten categories, the DGWR proved to be more robust to the multi-class setup than classical architectures due to the neurogenesis property that allows the network to retain the previously learned characteristics.
![](Results.png)

## <a id=ccl></a>Conclusion

In this paper, the authors developped a pretty old and unusual Neural Network architecture developped to reproduce the biological mecanism of neurogenesis. They were able to implement a complex deep network from this idea and they showed that its unique properties made this algorithm able to outperform classical DNN algorithms.

## <a id=biblio></a>Bibliography

Introduction of GWR :<br/>
Marsland, S., Shapiro, J., and Nehmzow, U. (2002). [A self-organising network that grows when required](https://seat.massey.ac.nz/personal/s.r.marsland/PUBS/NN02.pdf). *Neural Networks*, 15(8–9):1041–1058

Application on temporal data :<br/>
Strickert, M., & Hammer, B. (2005). [Merge SOM for temporal data](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.84.4929&rep=rep1&type=pdf). *Neurocomputing*, 64.

Description of multi-layers GWR :<br/>
German I. Parisi, Jun Tani, Cornelius Weber, Stefan Wermter. (2017). [Lifelong learning of human actions with deep neural network self-organization](https://reader.elsevier.com/reader/sd/pii/S0893608017302034?token=FD1B498D2F74836AE5D21F2594B14FF4AD9159867FDB012776AFBE28FCE8F613D0ABF796BC84D1B52D0CC6F4F0E20539). *Neural Networks*, 96:137–149.

Investigating the role of neurogenesis and presenting results on a new dataset :<br/>
German I. Parisi, Xu Ji, Stefan Wermter. (2018). [On the role of neurogenesis in overcomingcatastrophic forgetting](https://arxiv.org/abs/1811.02113). *arXiv*:1811.02113